In [22]:
import os, glob

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable, Derive, Variable, defineNumberingOfElementaryExpressions
from biogeme.models import logit, loglogit, piecewiseFormula, nested
from biogeme.models import lognested
from biogeme.results import bioResults, pickle

In [27]:
df = pd.read_table("new_lpmc02.dat", index_col=0)
df["dur_public"] = df["dur_pt_access"] + df["dur_pt_rail"] + df["dur_pt_bus"] + df["dur_pt_int"]
df["cost_driving"] = df["cost_driving_fuel"] + df["cost_driving_ccharge"]
df["cost_public"] = df["cost_transit"]

database = db.Database("LPMC",df)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

In [44]:
df["dur_public"].describe()

count    5e+03
mean     0.473
std      0.316
min     0.0111
25%      0.228
50%      0.392
75%      0.648
max       2.61
Name: dur_public, dtype: float64

In [28]:
# Model

  
# Choice
chosenAlternative = travel_mode


#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

CarOwn_2 = Beta('CarOwn_2',0,None,None,0)
CarOwn_3 = Beta('CarOwn_3',0,None,None,0)
CarOwn_4 = Beta('CarOwn_4',0,None,None,0)

LAMBDA = Beta('LAMBDA',1,None,None,0)

# parameters relevant to the nests
N_SM = Beta('N_SM',1,1,None, 0)
N_MOTOR = Beta('N_MOTOR',1,1,None, 0)


# socio-economic factors (interacting with Time)
Time_Age_1 = Beta('Time_Age_1', 0, None, None, 0)
Time_Age_2 = Beta('Time_Age_2', 0, None, None, 0)
Time_Age_3 = Beta('Time_Age_3', 0, None, None, 0)
Time_Age_4 = Beta('Time_Age_4', 0, None, None, 0)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving


#cost_public = DefineVariable('cost_public', cost_transit ,database)
#dur_public = DefineVariable('dur_public', (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int),database)
#cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)

#cost_public = Variable("cost_transit")
#dur_public = Variable("dur_pt_access") + Variable("dur_pt_rail") + Variable("dur_pt_bus") +\
#            Variable("dur_pt_int")
#cost_driving = Variable("cost_driving_fuel") + Variable("cost_driving_ccharge")

Opt1 = Constant1 + Total_TT1 * ((dur_walking) ** LAMBDA - 1)/LAMBDA + Time_Age_1 * dur_walking * age
Opt2 = Constant2 + Total_TT2 * ((dur_cycling) ** LAMBDA - 1)/LAMBDA+ CarOwn_2 * car_ownership +\
                    Time_Age_2 * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * (dur_public ** LAMBDA - 1)/LAMBDA + CarOwn_3 *\
                    car_ownership +\
                    Time_Age_3 * dur_public * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * ((dur_driving) ** LAMBDA - 1)/LAMBDA +\
                    CarOwn_4 * car_ownership + Time_Age_4 * dur_driving * age


V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1, 2: 1, 3: 1, 4: 1}


#Definitions of nests
N_SM = N_SM, [1, 2]
N_MOTOR = N_MOTOR, [3, 4]

nests = N_SM, N_MOTOR

In [30]:
sum_weights = database.data['Weights'].sum()
S = database.getSampleSize()
sample_normalized_weight = Weights * S / sum_weights

In [32]:
# value of time for car and public
VOT_public = Derive(Opt3, "dur_public")/Derive(Opt3, "cost_public")
VOT_car = Derive(Opt4, "dur_driving")/Derive(Opt4, "cost_driving")

simulate = {'weight': sample_normalized_weight,
            'VOT public': VOT_public,
            'VOT car': VOT_car,
            'Weighted VOT public': sample_normalized_weight*VOT_public,
            'Weighted VOT car': sample_normalized_weight*VOT_car,
           }

output_dir = "./model-nested-output"
filepath = os.path.join(output_dir, "logit_nested_lpmc_sm_motor")
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)
    
# # delete previously saved html and pickle
# for file in glob.glob(f"{filepath}*"):
#     os.remove(file)

biogeme  = bio.BIOGEME(database, simulate)
biogeme.modelName = "logit_nested_wtp"

betas = biogeme.freeBetaNames
results = bioResults(pickleFile=f"{filepath}.pickle")

beta_values = results.getBetaValues()

In [34]:
simulated_values = biogeme.simulate(beta_values)

wtp_public = simulated_values['VOT public'].mean() 
wtp_car = simulated_values['VOT car'].mean()

wtp_public, wtp_car

(39.15268102181185, 95.90853036592617)

In [46]:
simulated_values.head(20) # sanity check

,weight,VOT public,VOT car,Weighted VOT public,Weighted VOT car
0,1.07,20.8,77.2,22.2,82.6
1,1.07,18.7,24.7,20,26.4
2,1.07,27,84.4,28.9,90.4
3,1.07,15.3,63.6,16.3,68
4,1.07,34.5,132,36.9,141
5,1.07,15.2,21.7,16.3,23.3
6,1.07,29.1,86.3,31.1,92.3
7,1.07,34.5,157,36.9,168
8,1.07,14.2,19,15.2,20.3
9,1.07,34.5,139,36.9,149


In [35]:
# conf interval
b = results.getBetasForSensitivityAnalysis(betas, size=100)
left, right = biogeme.confidenceIntervals(b, 0.9)

lst_wtps = [wtp_public, wtp_car]
wtp_names = ["VOT public", "VOT car"]
print("Average (unweighted) values of time:\n")
for i in range(len(lst_wtps)):
    l = left[f"{wtp_names[i]}"].mean()
    r = right[f"{wtp_names[i]}"].mean()
    print(f"{wtp_names[i]}: {lst_wtps[i]:.2f} ({l:.2f}, {r:.2f})")

Average (unweighted) values of time:

VOT public: 39.15 (28.92, 55.96)
VOT car: 95.91 (66.56, 130.77)


In [36]:
df.head()["travel_mode"], df.columns # sanity check

(0    3
 1    3
 2    4
 3    4
 4    1
 Name: travel_mode, dtype: int64,
 Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
        'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
        'travel_year', 'travel_month', 'travel_date', 'day_of_week',
        'start_time', 'age', 'female', 'driving_license', 'car_ownership',
        'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
        'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
        'dur_driving', 'cost_transit', 'cost_driving_fuel',
        'cost_driving_ccharge', 'driving_traffic_percent', 'Weights',
        'dur_public', 'cost_driving', 'cost_public'],
       dtype='object'))